In [1]:
from PIL import Image
import os
import cv2
import shutil
import numpy as np

## Rename imágenes originales

In [1]:
# Esta funcion se encarga de renombrar las imagenes originales de 0 a 399
# 
# Argumentos:
#      - carpeta_origen: indica donde se encuentran las imagenes que necesitamos renombrar
#      - carpeta_destino: indica donde queremos que se almacenen las imagenes renombradas
#

def rename_img(carpeta_origen, carpeta_destino):
    
    # Crea la carpeta de destino si no existe
    if not os.path.exists(carpeta_destino):
        os.makedirs(carpeta_destino)

    # Lista de nombres de archivos en la carpeta de origen
    archivos = sorted(os.listdir(carpeta_origen))

    # Contador para el nuevo nombre
    contador = 0

    # Iterar sobre los archivos y renombrarlos, de forma consecutiva
    for archivo in archivos:
        if archivo.endswith((".jpg", ".png")):
            nuevo_nombre = f"{contador:03d}.png"
            ruta_origen = os.path.join(carpeta_origen, archivo)
            ruta_destino = os.path.join(carpeta_destino, nuevo_nombre)
            shutil.copy(ruta_origen, ruta_destino)
            contador += 1

    print("Imágenes renombradas y copiadas exitosamente.")

## Redimensión y mapeo

In [2]:
# Esta funcion lleva a cabo las tareas previas a la apliacion de tecnicas de DataAugmentation.
# Redimensiona las imagenes a un formato apto para que sirva como entrada del modelo 256x256 y realiza el mapeo correspondiente en las imagenes Ground Truth
# Argumentos:
#      - origen_images: indica donde se encuentran las imagenes originales
#      - origen_label: indica donde se encuentran las imagenes Ground Truth
#      - idx_train: lista indices para generar el conjunto de train
#      - idx_test: lista indices para generar el conjunto de test
#      - dest_train_images: ruta donde se almacenan las imagenes de train
#      - dest_train_label: ruta donde se almacenan las imagenes Ground Truth de train
#      - dest_test_images:ruta donde se almacenan las imagenes de test
#      - dest_test_label: ruta donde se almacenan las imagenes Ground Truth de test
#      - map_pixels_intermediate: diccionario de mapeo de los pixeles intermedio
#      - map_pixels_final: diccionario de mapeo de los pixeles final
#

def process_images(origen_images, origen_label, idx_train, idx_test, dest_train_images, dest_train_label, dest_test_images, dest_test_label, map_pixels_intermediate, map_pixels_final):
    
    # Creamos las rutas si no existen
    for carpeta_destino in [dest_train_images, dest_train_label, dest_test_images, dest_test_label]:
        if not os.path.exists(carpeta_destino):
            os.makedirs(carpeta_destino)

    # ~~~~~~~~~~~ GENERA CONJUNTOS TRAIN Y TEST ~~~~~~~~~~~
    # Si el indice esta en train, va para la carpeta train sino, para test
    for i in range(400):
        if i in idx_train:
            dest_images = dest_train_images
            dest_labels = dest_train_label
        elif i in idx_test:
            dest_images = dest_test_images
            dest_labels = dest_test_label
        else:
            continue

    # ~~~~~~~~~~~ PREPROCESA LAS IMAGENES ~~~~~~~~~~~
    
        # Redimensionar y guardar imagen original
        img = Image.open(os.path.join(origen_images, f"{i:03d}.png"))
        img = img.resize((256,256), Image.NEAREST)
        img.save(os.path.join(dest_images,f"{i:03d}.png"))

        mask = Image.open(os.path.join(origen_label,f"{i:03d}.png"))
        
        # Tenemos que convertir a array para poder convertir los valores de los pixeles
        mask_np = np.array(mask)

        # Mapea las imagenes Ground Truth
        for i,j in map_pixels_intermediate.items():
            # Sustituimos por los valores
            mask_np[mask_np==i] = j

        for i,j in map_pixels_final.items():
            # Sustituimos por los valores
            mask_np[mask_np==i] = j

        # Converitmos de array a imagen PIL
        mask_remap = Image.fromarray(mask_np)
        mask_remap = mask_remap.resize((256,256), Image.NEAREST)
        mask_remap.save(os.path.join(dest_labels,f"{i:03d}.png"))

## Data Augmentation 

In [3]:
# Esta funcion realiza las tecnicas de Data Augmentation sobre las imagenes preprocesadas
# Argumentos:
#      - image_dir: tiene la ruta a las imágenes
#      - is_mask: booleano indica si es una imagen Ground Truth
#

def transform_images(image_dir, is_mask):
    
    # Lista de ángulos para la rotación
    angulos = [45, 90, 135, 180, 225, 270, 315]

    # Recorre todas las imágenes en el directorio dado
    for filename in os.listdir(image_dir):
        if filename.endswith(".png"):
            
            # Obtenemos el numero de la imagen
            number = filename.split(".")[0]
            
            # Carga imagen
            img = cv2.imread(os.path.join(image_dir, filename))

            # Crea y guarda imagen espejo
            img_esp = cv2.flip(img, 1)
            cv2.imwrite(os.path.join(image_dir, f"{number}_mirror.png"), img_esp)

            # Crea y guarda imágenes rotadas
            for angulo in angulos:
                
                # Si es mascara aplicamos INERT_NEAREST para no variar los valores de los pixeles (pertenecen a clases) y elige el pixel mas cercano
                # Si no lo es aplicamos INTER_LINEAR hace que la redimension sea mas suave y con detalle
                if is_mask:
                    interpolation = cv2.INTER_NEAREST
                else:
                    interpolation = cv2.INTER_LINEAR
                
                # Generamos una matriz M de rotacion, respecto al centro de la imagen img
                M = cv2.getRotationMatrix2D((int(img.shape[1]/2), int(img.shape[0]/2)), angulo, 1)

                # Creamos la imagen rotada, aplicando la matriz de rotacion a la imagen original
                img_rot = cv2.warpAffine(img, M, (img.shape[1], img.shape[0]), flags=interpolation, borderMode=cv2.BORDER_REFLECT)

                # Guardamos la imagen original rotada
                cv2.imwrite(os.path.join(image_dir, f"{number}_rot{angulo}.png"), img_rot)

                M_mirror = cv2.getRotationMatrix2D((int(img_esp.shape[1]/2), int(img_esp.shape[0]/2)), angulo, 1)
                img_esp_rot = cv2.warpAffine(img_esp, M_mirror, (img_esp.shape[1], img_esp.shape[0]), flags=interpolation, borderMode=cv2.BORDER_REFLECT)

                # Ahora se ha aplicado la rotacion a la imagen espejo
                cv2.imwrite(os.path.join(image_dir, f"{number}_mirror_rot{angulo}.png"), img_esp_rot)

                # Ajustar brillo y contraste
                if is_mask == False: 
                    img_bc = cv2.convertScaleAbs(img, alpha=1.3, beta=10)
                    img_esp_bc = cv2.convertScaleAbs(img_esp, alpha=1.3, beta=10)
                    
                # si es una Ground Truth no aplicamos cambios, para no variar los valores de los pixeles
                elif is_mask == True:
                    img_bc = img
                    img_esp_bc = img_esp
                
                # Almacenamos las imagenes variadas

                # Imagen original
                cv2.imwrite(os.path.join(image_dir, f"{number}_bc.png"), img_bc)
                        
                # Imagen espejo
                cv2.imwrite(os.path.join(image_dir, f"{number}_mirror_bc.png"), img_esp_bc)

                # Crear y guardar imágenes rotadas para la imagen con brillo y contraste ajustados
                for angulo in angulos:
                    
                    # Genero la matriz de rotacion para las imagenes con el brillo y contraste ajustados
                    M = cv2.getRotationMatrix2D((int(img_bc.shape[1]/2), int(img_bc.shape[0]/2)), angulo, 1)
                    
                    img_bc_rot = cv2.warpAffine(img_bc, M, (img_bc.shape[1], img_bc.shape[0]),flags=interpolation, borderMode=cv2.BORDER_REFLECT)
                    
                    cv2.imwrite(os.path.join(image_dir,f"{number}_bc_rot{angulo}.png"),img_bc_rot)

                    # Igual para las imagenes espejo
                    M_mirror_bc = cv2.getRotationMatrix2D((int(img_esp_bc.shape[1] / 2),int(img_esp_bc.shape[0] / 2)),angulo,1)
                    
                    img_esp_bc_rot = cv2.warpAffine(img_esp_bc,M_mirror_bc,(img_esp_bc.shape[1],img_esp_bc.shape[0]),flags=interpolation,borderMode=cv2.BORDER_REFLECT)
                    
                    cv2.imwrite(os.path.join(image_dir,f"{number}_mirror_bc_rot{angulo}.png"),img_esp_bc_rot)

## Comprobamos concordancia de nombres despues de las transformaciones

In [4]:
# Comprobamos correlacion de archivos imagen - mascara
def compare_mask_img(dir1, dir2):
    
    # Obtenemos el conjutno de archivos de cada directorio
    archivos_1 = set(os.listdir(dir1))
    archivos_2 = set(os.listdir(dir2))
    
    return archivos_1 == archivos_2